# [Selected Problems in Machine Learning (NPFL097)](https://ufal.mff.cuni.cz/courses/npfl097)

In [59]:
import numpy as np
from collections import Counter

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import random
from glob import glob
from tqdm import tqdm, tqdm_notebook

In [10]:
with open("data/eng-input.txt", "r") as f:
    data = f.read().replace("\n", "")
    
N = len(data)

In [42]:
s = np.random.randint(0, 2, size=N-2)

In [43]:
def cut_words(data, s):
    s_idx = np.where(s)[0] + 1

    idx_start = np.hstack([np.array([0]), s_idx])
    idx_end = np.hstack([s_idx, np.array([N])])
            
    words = [data[start:end] for start, end in zip(idx_start, idx_end)]
    
    return words

In [41]:
cut_words(data, s)

['o',
 'ral',
 't',
 'ernativ',
 'ely',
 'a',
 's',
 'o',
 'p',
 'posi',
 'nga',
 'utho',
 'r',
 'i',
 't',
 'y',
 'i',
 'n',
 'the',
 'con',
 'd',
 'uctofhumanrel',
 'at',
 'io',
 'n',
 's',
 'pr',
 'op',
 'one',
 'n',
 'ts',
 'o',
 'fa',
 'n',
 'arc',
 'hi',
 's',
 'mkno',
 'wn',
 'a',
 'sa',
 'narchistsadv',
 'oca',
 't',
 'es',
 't',
 'at',
 'ele',
 'sss',
 'ociet',
 'i',
 'e',
 's',
 'as',
 't',
 'he',
 'o',
 'nlym',
 'o',
 'r',
 'al',
 'f',
 'o',
 'rmof',
 'soci',
 'a',
 'lor',
 'g',
 'a',
 'n',
 'iza',
 'tionth',
 'ere',
 'ar',
 'eman',
 'yt',
 'y',
 'pe',
 'sa',
 'n',
 'dtr',
 'ad',
 'i',
 't',
 'io',
 'nsof',
 'anarc',
 'h',
 'ism',
 'not',
 'a',
 'll',
 'ofw',
 'hich',
 'a',
 'r',
 'em',
 'u',
 't',
 'ual',
 'lye',
 'x',
 'clu',
 'sivea',
 'n',
 'ar',
 'c',
 'h',
 'is',
 't',
 'sch',
 'o',
 'o',
 'l',
 'soft',
 'hought',
 'c',
 'an',
 'd',
 'iff',
 'e',
 'r',
 'f',
 'u',
 'n',
 'd',
 'a',
 'me',
 'nt',
 'a',
 'l',
 'lysuppo',
 'rt',
 'in',
 'g',
 'a',
 'ny',
 'th',
 'i',
 'ng

In [85]:
alpha = 5
p_c = 0.8

s = np.random.randint(0, 2, size=N - 2)

num_iter = 20
words = cut_words(data, s)

count = Counter(words)

def P0(w):
    U = 1.0/5
    
    return U**len(w)
    # return U**len(w) * p_c**(len(w)-1)

for iter in range(num_iter):
    for i in tqdm_notebook(np.random.permutation(N - 2)):
        prev_left = 0
        for j in reversed(range(i)):
            if s[j] == 1:
                prev_left = j
                break
                
        next_right = len(s)
        for j in range(i + 1, len(s)):
            if s[j] == 1:
                next_right = j
                break
            
        prev_word = data[prev_left:i]
        next_word = data[i:next_right]
        joined_word = prev_word + next_word
        
        if s[i] == 0:
            count[joined_word] = max(0, count[joined_word] - 1)
            i -= 1
        else:
            count[prev_word] = max(0, count[prev_word] - 1)
            count[next_word] = max(0, count[next_word] - 1)
            i -= 2
    
        i = max(0, i)
            
        p0 = (alpha * P0(joined_word) + count[joined_word]) / (alpha + i)
        
        a = (alpha * P0(prev_word) + count[prev_word]) / (alpha + i)
        b = (alpha * P0(next_word) + count[next_word]) / (alpha + i + 1)
        p1 = a * b
        
        norm = p0 + p1
        p0 = p0 / norm
        p1 = p1 / norm
        
        s[i] = np.random.choice([0, 1], p=[p0, p1])
        
        if s[i] == 0:
            count[joined_word] += 1
            i += 1
        else:
            count[prev_word] += 1
            count[next_word] += 1
            i += 2

In [86]:
cut_words(data, s)

['or',
 'a',
 'l',
 't',
 'e',
 'r',
 'na',
 'tiv',
 'e',
 'l',
 'y',
 'a',
 'sop',
 'p',
 'os',
 'in',
 'g',
 'a',
 'u',
 'tho',
 'r',
 'i',
 't',
 'y',
 'in',
 'th',
 'e',
 'co',
 'n',
 'd',
 'u',
 'c',
 'to',
 'f',
 'hu',
 'm',
 'a',
 'nr',
 'elatio',
 'n',
 'sp',
 'r',
 'opo',
 'n',
 'e',
 'nts',
 'of',
 'a',
 'n',
 'a',
 'r',
 'ch',
 'i',
 'sm',
 'k',
 'n',
 'o',
 'wna',
 'sa',
 'n',
 'a',
 'rch',
 'i',
 's',
 't',
 's',
 'a',
 'dvocate',
 'st',
 'at',
 'el',
 'e',
 's',
 's',
 'soci',
 'et',
 'ie',
 's',
 'a',
 's',
 'theon',
 'l',
 'y',
 'm',
 'o',
 'ra',
 'lf',
 'o',
 'r',
 'm',
 'of',
 'so',
 'ci',
 'a',
 'l',
 'or',
 'ga',
 'n',
 'i',
 'z',
 'at',
 'ion',
 't',
 'herearem',
 'any',
 'ty',
 'p',
 'es',
 'andtra',
 'dition',
 's',
 'ofan',
 'a',
 'rchi',
 's',
 'm',
 'not',
 'all',
 'ofw',
 'hich',
 'a',
 'r',
 'emu',
 'tu',
 'allye',
 'x',
 'cl',
 'usi',
 'vea',
 'n',
 'a',
 'r',
 'chist',
 's',
 'cho',
 'olsof',
 'th',
 'ough',
 't',
 'c',
 'an',
 'd',
 'iff',
 'e',
 'rfun',
